# Build train and test matrices

In [1]:
import pandas as pd
import numpy as np
import feather

df = (feather.read_dataframe('/home/SHARED/SOLAR/data/oahu_min.feather')
             .set_index('Datetime'))

In [2]:
# https://stackoverflow.com/questions/15722324/sliding-window-in-numpy
def window_stack_forward(a, stepsize=1, width=3):
    return np.hstack( a[i:1+i-width or None:stepsize] for i in range(0, width) )

In [3]:
# I feel this function can also be done for pd.DataFrame
def window_stack(a, width=3):
    n = a.shape[0]
    return np.hstack(list(a[(width-1-i):(n-i)] for i in range(0, width)))

In [4]:
# In pandas 0.24, use df.to_numpy() instead of df.values. Also care with non-numeric columns
width = 61
a = window_stack(df.values, width=width)

In [5]:
times   = [ ('t' if not idx else 't-{:d}'.format(idx)) for idx in range(width) ]
columns = pd.MultiIndex.from_product((times, df.columns), names=('time', 'location'))

In [6]:
# Convert back to DataFrame, just for convenience of having indexes
df_roll = pd.DataFrame(a, index=df.index[width-1:], columns=columns)

In [7]:
# Split target (time t) and variables (times t-1 to t-width+1)
y = df_roll['t']
X = df_roll.drop(columns='t', level='time')

In [8]:
# Split train-test, approximately 12 and 4 months respectively
X_train, X_test = X[:'2011-07-31'], X['2011-08-01':]
y_train, y_test = y[:'2011-07-31'], y['2011-08-01':]

In [9]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(449885, 1140)
(82892, 1140)
(449885, 19)
(82892, 19)


# Convolutional predictor

First we preprocess the dataset (for the moment, we'll just use as features the t-1 values at each sensor)

In [10]:
# We only use the previous timestep as features
X_tr1 = X_train['t-1']
y_tr1 = y_train

X_te1 = X_test['t-1']
y_te1 = y_test

Now, in order to use a 1D convolution, we are going to sort the sensors. For the initial test, we'll just sort them by longitude (from East to West). That way, nearer sensors are in close positions in the tensor, so the 1D convolution may extract useful correlations.

Note: many other possible ordenations of the sensors could be added as new channels in the input tensor

In [11]:
# We load the info of the sensors to extract the longitude information
info = pd.read_csv('/home/SHARED/SOLAR/data/info.csv')

info.Location = info.Location.apply(lambda x: (x[:2] + x[-2:]).replace('_', ''))
info.index = info.Location
# Sorted longitudes
longs = info['       Longitude'].sort_values(ascending=False)

# We drop two sensors (they are different compared to the other 17, since they are "tilted")
X_tr1.drop('GT_AP6', inplace=True, axis=1)
y_tr1.drop('GT_AP6', inplace=True, axis=1)
X_tr1.drop('GT_DH1', inplace=True, axis=1)
y_tr1.drop('GT_DH1', inplace=True, axis=1)
X_te1.drop('GT_AP6', inplace=True, axis=1)
y_te1.drop('GT_AP6', inplace=True, axis=1)
X_te1.drop('GT_DH1', inplace=True, axis=1)
y_te1.drop('GT_DH1', inplace=True, axis=1)

# Just some auxiliar code to homogeneize name of sensors across different tables
homogen_name = lambda x: x[-4:].replace('_', '')
X_tr1.columns = [homogen_name(x) for x in X_tr1.columns.values.tolist()]
y_tr1.columns = [homogen_name(x) for x in y_tr1.columns.values.tolist()]
X_te1.columns = [homogen_name(x) for x in X_te1.columns.values.tolist()]
y_te1.columns = [homogen_name(x) for x in y_te1.columns.values.tolist()]


# Finally, we sort the data according to sensor's longitude
X_tr1_1 = X_tr1[longs.index]
y_tr1_1 = y_tr1[longs.index]
X_te1_1 = X_te1[longs.index]
y_te1_1 = y_te1[longs.index]

/home/victor/miniconda3/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


Now we specify which sensor do we want to predict and test.

(In the future, we need to discuss how are we going to predict, if just by looping over each sensor, or just give a vectorial prediction)

In [14]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Lambda, Reshape, Add, Multiply, Subtract, Dropout
from keras.layers import Conv2D, MaxPooling2D, LocallyConnected1D, Conv1D, UpSampling1D, MaxPooling1D, Dot, Concatenate

from keras import backend as K

Using TensorFlow backend.


Model architecture is defined below.

Some highlights:
* Locally connected works better than pure convolutional at the first layers (probably because the sensors at not located in a uniform grid)
* Trick to improve acc: add a final layer combining the convolutional prediction with the persistance prediction, so in case the input is "strange", the model could learn to output the persistance prediction (i.e., the previous time-step), which is somewhat reasonable

In [16]:
def make_model_sensor(index_sensor, n_sensors=17):
    ''' Returns a model using all the sensors to predict index_sensor '''
    xin = Input(shape=(n_sensors,1), name='main_input')
    x = LocallyConnected1D(8, 7, data_format = 'channels_last', padding='valid')(xin)
    x = Activation('relu')(x)
    x = LocallyConnected1D(16, 5, data_format = 'channels_last', padding='valid')(x)
    x = Activation('relu')(x)
    x = Conv1D(32, 3, data_format = 'channels_last', padding='causal')(x)
    xl = Flatten()(x)
    xl = Dropout(0.2)(xl)
    xo = Dense(1)(xl)

    # use date info here?
    xinf = Flatten()(xin)
    s  = Dense(5)(xinf)
    s = Activation('tanh')(s)
    s = Dense(2)(s)
    s = Activation('softmax')(s)


    xin_0 = Activation('relu')(xin)
    xin_1 = Lambda(lambda x : x[:,index_sensor,:])(xin_0)
    xo_m = Dot(axes=1)([Concatenate()([xo,xin_1]), s])
    # instead of this, use a residual connection

    #xom = Add()([xo, xin_1])

    # better than just residual (xom)
    xo_m = Activation('relu')(xo_m)

    model = Model(inputs=[xin], outputs=[xo_m])
    return model

In [126]:
lr = 0.0001
lr = 0.0001
opt = keras.optimizers.Adam(lr=lr)

# We add a callback to log metrics and another one to schedule the learning rate

#see clr.py in this same folder
from clr import CyclicLR

c1 = keras.callbacks.BaseLogger(stateful_metrics=None)
c2 = CyclicLR(step_size=250, base_lr=lr)
c3 = keras.callbacks.History()

Now we are ready to train. The below configuration should take 2 minutes in a 16 core CPU
(no GPU needed). We are using a huge batch-size to speed up things

In [127]:
n_sensors = 17
sensor = 'AP5'  # 'AP5' initial
index_sensor = 4

def make_features(sensor='AP5'):
    X_tr1_1_np = X_tr1_1.values
    y_tr1_1_np = y_tr1_1[sensor].values

    X_te1_1_np = X_te1_1.values
    y_te1_1_np = y_te1_1[sensor].values
    return X_tr1_1_np, y_tr1_1_np, X_te1_1_np, y_te1_1_np

In [128]:
batch_size = 1 << 11   # as big as possible so we can explore many models
epochs = 1 << 5

In [129]:
# Final MAE for AP5 sensor is around 40.2
# Other sensors are also around 40-50..

In [130]:
def train_and_test_sensor(id_sensor=4):
    longs_np = longs.index.values
    X_tr1_1_np, y_tr1_1_np, X_te1_1_np, y_te1_1_np = make_features(sensor=longs_np[id_sensor])
    model = make_model_sensor(id_sensor, n_sensors=17)
    model.compile(opt, loss='mean_absolute_error')
    model.fit(np.atleast_3d(X_tr1_1_np), y_tr1_1_np, batch_size=batch_size, epochs=epochs, validation_data=
          (np.atleast_3d(X_te1_1_np),y_te1_1_np), callbacks=[c1, c2, c3])
    return longs_np[id_sensor], c3.history['val_loss'][-1]
    # Note: for this quick trial, we are using as validation the test!! 

In [ ]:
maes = {}
for i in range(len(longs_np)):
    print(i, longs_np[i])
    sensor, mae = train_and_test_sensor(i)
    maes[sensor] = mae 

0 AP7
Train on 449885 samples, validate on 82892 samples
Epoch 1/32
 14336/449885 [..............................] - ETA: 54s - loss: 82.6605 

/home/victor/miniconda3/lib/python3.6/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.300463). Check your callbacks.
  % delta_t_median)


449885/449885 [==============================] - 8s 18us/step - loss: 62.0225 - val_loss: 57.2572
Epoch 2/32
449885/449885 [==============================] - 5s 11us/step - loss: 59.2373 - val_loss: 56.9693
Epoch 3/32
449885/449885 [==============================] - 6s 14us/step - loss: 59.1477 - val_loss: 56.9432
Epoch 4/32
449885/449885 [==============================] - 6s 13us/step - loss: 59.1216 - val_loss: 56.9722
Epoch 5/32
449885/449885 [==============================] - 5s 12us/step - loss: 59.0265 - val_loss: 56.8951
Epoch 6/32
449885/449885 [==============================] - 6s 14us/step - loss: 59.0398 - val_loss: 56.7696
Epoch 7/32
449885/449885 [==============================] - 5s 12us/step - loss: 58.9424 - val_loss: 56.7727
Epoch 8/32
449885/449885 [==============================] - 5s 12us/step - loss: 58.9254 - val_loss: 57.0870
Epoch 9/32
449885/449885 [==============================] - 6s 13us/step - loss: 58.9006 - val_loss: 56.7262
Epoch 10/32
449885/449885 [===

Epoch 11/32
449885/449885 [==============================] - 5s 12us/step - loss: 52.2173 - val_loss: 52.7729
Epoch 12/32
449885/449885 [==============================] - 6s 13us/step - loss: 52.1129 - val_loss: 52.8408
Epoch 13/32
449885/449885 [==============================] - 5s 12us/step - loss: 52.1177 - val_loss: 52.5343
Epoch 14/32
449885/449885 [==============================] - 6s 13us/step - loss: 51.9374 - val_loss: 52.6950
Epoch 15/32
449885/449885 [==============================] - 6s 13us/step - loss: 51.9738 - val_loss: 52.5467
Epoch 16/32
449885/449885 [==============================] - 5s 11us/step - loss: 51.7626 - val_loss: 52.1049
Epoch 17/32
449885/449885 [==============================] - 6s 13us/step - loss: 51.9115 - val_loss: 52.0424
Epoch 18/32
449885/449885 [==============================] - 6s 13us/step - loss: 51.6568 - val_loss: 52.0198
Epoch 19/32
449885/449885 [==============================] - 5s 11us/step - loss: 51.6815 - val_loss: 52.1560
Epoch 20/3

Epoch 21/32
449885/449885 [==============================] - 6s 12us/step - loss: 44.2361 - val_loss: 40.9814
Epoch 22/32
449885/449885 [==============================] - 6s 12us/step - loss: 44.0844 - val_loss: 40.8825
Epoch 23/32
449885/449885 [==============================] - 6s 13us/step - loss: 44.1257 - val_loss: 41.3094
Epoch 24/32
449885/449885 [==============================] - 6s 12us/step - loss: 44.1115 - val_loss: 40.9257
Epoch 25/32
449885/449885 [==============================] - 6s 13us/step - loss: 44.0013 - val_loss: 40.9864
Epoch 26/32
449885/449885 [==============================] - 6s 13us/step - loss: 44.2495 - val_loss: 40.8569
Epoch 27/32
449885/449885 [==============================] - 5s 12us/step - loss: 43.9804 - val_loss: 40.9677
Epoch 28/32
449885/449885 [==============================] - 6s 13us/step - loss: 44.1350 - val_loss: 41.1542
Epoch 29/32
449885/449885 [==============================] - 6s 13us/step - loss: 43.9751 - val_loss: 40.7533
Epoch 30/3

449885/449885 [==============================] - 6s 13us/step - loss: 42.3771 - val_loss: 42.9830
Epoch 8/32
449885/449885 [==============================] - 6s 14us/step - loss: 41.6436 - val_loss: 42.7469
Epoch 9/32
449885/449885 [==============================] - 5s 12us/step - loss: 41.8047 - val_loss: 42.5463
Epoch 10/32
449885/449885 [==============================] - 6s 13us/step - loss: 41.4442 - val_loss: 42.3194
Epoch 11/32
449885/449885 [==============================] - 6s 14us/step - loss: 41.5065 - val_loss: 42.4468
Epoch 12/32
449885/449885 [==============================] - 5s 12us/step - loss: 41.3945 - val_loss: 42.0062
Epoch 13/32
415744/449885 [==========================>...] - ETA: 0s - loss: 41.3350

In [134]:
maes = pd.Series(maes, name='MAE').sort_values()

In [135]:
maes

AP3      9.689523
DH8     39.415958
DH11    41.442673
AP5     41.738685
DH6     42.791773
DH10    43.391440
DH9     43.446805
DH7     46.676140
AP1     47.536166
DH3     47.866472
DH4     48.048144
DH2     50.818569
AP4     51.337906
DH1     51.957808
DH5     52.437509
AP7     56.508694
AP6     57.921463
Name: MAE, dtype: float64